## import & load google drive

In [ ]:
import pandas as pd
import numpy as np

# google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# config

In [ ]:
path = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/sp500/MMM/MMM_OHLC_1d.csv"

# read data with pandas.DataFrame

In [ ]:
raw_df = pd.read_csv(path)
raw_df.head(10)

,Date,Open,High,Low,Close,Volume
0,2017-01-03 00:00:00-05:00,149.337611,150.314655,147.993130,148.686249,2509300
1,2017-01-04 00:00:00-05:00,148.669517,149.396033,148.318784,148.911697,1542000
2,2017-01-05 00:00:00-05:00,148.861582,149.596458,147.717525,148.402298,1447800
3,2017-01-06 00:00:00-05:00,148.051524,149.145491,146.807261,148.836502,1625000
4,2017-01-09 00:00:00-05:00,148.953371,148.961730,147.976328,148.034790,1622600
5,2017-01-10 00:00:00-05:00,148.109996,148.218560,147.233158,147.458633,2030100
6,2017-01-11 00:00:00-05:00,147.500384,149.020225,147.299960,148.552582,1579600
7,2017-01-12 00:00:00-05:00,147.784299,148.393905,146.765499,148.176788,1321800
8,2017-01-13 00:00:00-05:00,148.293738,148.569317,147.667427,148.135071,1265500
9,2017-01-17 00:00:00-05:00,147.809357,148.377207,147.183047,148.026474,1557500


In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1447 non-null   object 
 1   Open    1447 non-null   float64
 2   High    1447 non-null   float64
 3   Low     1447 non-null   float64
 4   Close   1447 non-null   float64
 5   Volume  1447 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 68.0+ KB


# convert ‘Date’ into datatime

In [ ]:
def drop_hours(row):
  row = row.split(' ')[0]
  return row

In [ ]:
raw_df['Date'] = raw_df['Date'].apply(drop_hours)

convert "Date" into datatime type

In [ ]:
raw_df['Date'] = pd.to_datetime(raw_df['Date'])

In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1447 non-null   datetime64[ns]
 1   Open    1447 non-null   float64       
 2   High    1447 non-null   float64       
 3   Low     1447 non-null   float64       
 4   Close   1447 non-null   float64       
 5   Volume  1447 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 68.0 KB


In [ ]:
display(raw_df.Date.min())
display(raw_df.Date.max())

Timestamp('2017-01-03 00:00:00')

Timestamp('2022-09-30 00:00:00')

create empty dataFrame

In [ ]:
df = pd.DataFrame(columns=['stock','weekNo','days','成交量均值','成交金额均值','上周涨跌幅','涨跌幅均值','股价波动程度','开收盘价方差','未来一周的收益率'])

# get features

In [ ]:
start_date = '2017-1-2'
end_date = '2022-9-30'

week = pd.Period('2017-1-2', freq='W')

last_week = float('inf')

In [ ]:
count = 0
while week.start_time < pd.to_datetime(end_date):

  record = dict()
  # record['start_time'] = week.start_time
  # record['end_time'] = week.end_time
  count += 1
  record['weekNo'] = count

  # get dates in current week
  mask = (raw_df['Date'] >= week.start_time) & (raw_df['Date'] < week.end_time)
  df_week = raw_df.loc[mask]

  # drop the index (important)
  df_week = df_week.reset_index(drop=True)

  # count days
  n_days = df_week.shape[0]
  record['days'] = df_week.shape[0]

  # 成交量均值
  record['成交量均值'] = df_week['Volume'].mean()

  # 成交金额均值
  record['成交金额均值'] = (df_week['Volume']*df_week['Close']).mean()

  # 上周涨跌幅 = (end..price - start.rice) / start.price
  if last_week == 'inf':
    record['上周涨跌幅'] = np.nan
  else:
    record['上周涨跌幅'] = last_week

  # 涨跌幅均值 = 涨跌幅/开盘日
  record['涨跌幅均值'] = ((df_week['Close'][df_week.shape[0]-1] - df_week['Close'][0]) / df_week['Close'][0]) / n_days
  last_week = (df_week['Close'][df_week.shape[0]-1] - df_week['Close'][0]) / df_week['Close'][0]

  # 股价波动程度 = 标准差/平均值
  record['股价波动程度'] = df_week['Close'].std() / df_week['Close'].mean()

  # 开盘价，收盘价方差 = 开盘价方差+收盘价方差
  record['开收盘价方差'] = df_week['Close'].var() + df_week['Open'].var()

  # 未来一周的收益率 = (一周后收盘价格-当前收盘价格)/当前收盘价
  next_week = week + 1
  mask = (raw_df['Date'] >= next_week.start_time) & (raw_df['Date'] < next_week.end_time)
  df_next_week = raw_df.loc[mask]
  df_next_week = df_next_week.reset_index(drop=True)
  cur_close = df_week['Close'][df_week.shape[0]-1]
  if df_next_week.shape[0] != 0:
    record['未来一周的收益率'] = (df_next_week['Close'][df_next_week.shape[0]-1] - cur_close) / cur_close

  record['stock'] = "AAPL"

  df = df.append([record],ignore_index=True)

  week += 1

In [ ]:
df

,stock,weekNo,days,成交量均值,成交金额均值,上周涨跌幅,涨跌幅均值,股价波动程度,开收盘价方差,未来一周的收益率
0,AAPL,1,4,1781025.0,2.648591e+08,inf,0.000253,0.001513,0.334140,-0.004713
1,AAPL,2,5,1563920.0,2.315071e+08,0.001011,0.000135,0.002666,0.461146,0.006201
2,AAPL,3,4,1647100.0,2.451644e+08,0.000677,0.001735,0.003663,1.124048,-0.005659
3,AAPL,4,5,2556440.0,3.776762e+08,0.006939,-0.001154,0.005375,0.886468,-0.013748
4,AAPL,5,5,2373760.0,3.466297e+08,-0.005770,-0.000433,0.002688,1.349654,0.022623
...,...,...,...,...,...,...,...,...,...,...
295,AAPL,296,5,31997600.0,4.004827e+09,-0.087930,-0.007577,0.014624,6.479938,0.011919
296,AAPL,297,4,30065950.0,3.602579e+09,-0.037884,0.013937,0.022563,12.641034,-0.052721
297,AAPL,298,5,8744500.0,1.055570e+09,0.055746,-0.012298,0.027933,25.124960,-0.031044
298,AAPL,299,5,3688500.0,4.242319e+08,-0.061489,-0.006259,0.013618,4.094778,-0.022037


In [ ]:
info  = pd.read_csv('/content/drive/MyDrive/BDT SEM 1/MSBD 5001/sp500/AAPL/AAPL_info.csv')
info = info.set_index('Attribute')

# Whole pipeline

In [1]:
import pandas as pd
import numpy as np
import os

# google drive
from google.colab import drive
drive.mount('/content/drive') 

# ROOT
root = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/sp500/"

# save
save = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/dataset"

# file name
file_name = "weekdate_2211171705.csv"

os.chdir(root)
os.listdir(root)

dirs = os.listdir()

Mounted at /content/drive


In [2]:
df = pd.DataFrame(columns=['name','stock','start_time','end_time','days','成交量均值','成交金额均值','上周涨跌幅','涨跌幅均值','股价波动程度','开收盘价方差','未来一周的收益率'])

In [3]:
def drop_hours(row):
  row = row.split(' ')[0]
  return row

In [4]:
def get_features(df, raw_df, name, stock):

  # start and end
  start_date = '2017-1-2'
  end_date = '2022-9-30'

  # first week
  week = pd.Period(start_date, freq='W')

  last_week = float('inf')  

  while week.start_time < pd.to_datetime(end_date):

    record = dict()

    record['start_time'] = week.start_time
    record['end_time'] = week.end_time

    # get dates in current week
    mask = (raw_df['Date'] >= week.start_time) & (raw_df['Date'] < week.end_time)
    df_week = raw_df.loc[mask]
    #if df_week.shape[0] == 0:
    #  week += 1
    #  continue

    # drop the index (important)
    df_week = df_week.reset_index(drop=True)

    # count days
    n_days = df_week.shape[0]
    record['days'] = df_week.shape[0]

    # 成交量均值
    record['成交量均值'] = df_week['Volume'].mean()

    # 成交金额均值
    record['成交金额均值'] = (df_week['Volume']*df_week['Close']).mean()

    # 上周涨跌幅 = (end..price - start.rice) / start.price
    if last_week == 'inf':
      record['上周涨跌幅'] = np.nan
    else:
      record['上周涨跌幅'] = last_week

    # 涨跌幅均值 = 涨跌幅/开盘日
    record['涨跌幅均值'] = ((df_week['Close'][df_week.shape[0]-1] - df_week['Close'][0]) / df_week['Close'][0]) / n_days
    last_week = (df_week['Close'][df_week.shape[0]-1] - df_week['Close'][0]) / df_week['Close'][0]

    # 股价波动程度 = 标准差/平均值
    record['股价波动程度'] = df_week['Close'].std() / df_week['Close'].mean()

    # 开盘价，收盘价方差 = 开盘价方差+收盘价方差
    record['开收盘价方差'] = df_week['Close'].var() + df_week['Open'].var()

    # 未来一周的收益率 = (一周后收盘价格-当前收盘价格)/当前收盘价
    next_week = week + 1
    mask = (raw_df['Date'] >= next_week.start_time) & (raw_df['Date'] < next_week.end_time)
    df_next_week = raw_df.loc[mask]
    df_next_week = df_next_week.reset_index(drop=True)
    cur_close = df_week['Close'][df_week.shape[0]-1]
    if df_next_week.shape[0] != 0:
      record['未来一周的收益率'] = (df_next_week['Close'][df_next_week.shape[0]-1] - cur_close) / cur_close

    record['stock'] = stock
    record['name'] = name

    df = df.append([record],ignore_index=True)

    week += 1

    df.to_csv(save + file_name, mode='a',header=False)
  return df
    

In [ ]:
dirs[0]

'MMM'

In [5]:
dirs = ['^GSPC']

for stock in dirs:
  path = os.path.join(root, stock)
  files = os.listdir(path)

  if stock+'_OHLC_1d.csv' not in files or stock+'_info.csv' not in files:
    continue
  #if stock == '^GSPC':
  #  continue

  # read
  raw_df = pd.read_csv(path+'/'+stock+'_OHLC_1d.csv')
  info  = pd.read_csv(path+'/'+stock+'_info.csv')

  if raw_df.shape[0] == 0:
    continue

  # Company Name
  info = info.set_index('Attribute')
  company_name_short = info.at['shortName','Value']

  # Convert to str to datetime
  raw_df['Date'] = raw_df['Date'].apply(drop_hours)
  raw_df['Date'] = pd.to_datetime(raw_df['Date'])

  if raw_df['Date'][0] > pd.to_datetime('2017-1-30'):
    continue

  print(stock, company_name_short)

  # features
  df_final = get_features(df, raw_df, company_name_short, stock)
  df_final.to_csv(save+'/'+stock+'_week_features.csv')

^GSPC S&P 500


In [6]:
df

,name,stock,start_time,end_time,days,成交量均值,成交金额均值,上周涨跌幅,涨跌幅均值,股价波动程度,开收盘价方差,未来一周的收益率


# Random Sample 30 stocks

In [ ]:
import pandas as pd
import numpy as np
import os

# google drive
from google.colab import drive
drive.mount('/content/drive') 

# ROOT
root = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/dataset/"

# save
save = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/"

os.chdir(root)
os.listdir(root)

dirs = os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dirs[:477]

['MMM_week_features.csv',
 'AOS_week_features.csv',
 'ABT_week_features.csv',
 'ABBV_week_features.csv',
 'ABMD_week_features.csv',
 'ACN_week_features.csv',
 'ATVI_week_features.csv',
 'ADM_week_features.csv',
 'ADBE_week_features.csv',
 'AAP_week_features.csv',
 'AMD_week_features.csv',
 'AES_week_features.csv',
 'AFL_week_features.csv',
 'A_week_features.csv',
 'APD_week_features.csv',
 'AKAM_week_features.csv',
 'ALK_week_features.csv',
 'ALB_week_features.csv',
 'ARE_week_features.csv',
 'ALGN_week_features.csv',
 'ALLE_week_features.csv',
 'LNT_week_features.csv',
 'ALL_week_features.csv',
 'GOOGL_week_features.csv',
 'GOOG_week_features.csv',
 'MO_week_features.csv',
 'AMZN_week_features.csv',
 'AMCR_week_features.csv',
 'AEE_week_features.csv',
 'AAL_week_features.csv',
 'AEP_week_features.csv',
 'AXP_week_features.csv',
 'AIG_week_features.csv',
 'AMT_week_features.csv',
 'AWK_week_features.csv',
 'AMP_week_features.csv',
 'ABC_week_features.csv',
 'AME_week_features.csv',
 'A

In [ ]:
from random import sample
sampled_list = sample(dirs, 30)
sampled_list = [i.split('_')[0] for i in sampled_list]
sampled_list

['AES',
 'DRI',
 'LVS',
 'ZION',
 'TJX',
 'KMX',
 'DVA',
 'PYPL',
 'INTU',
 'ADI',
 'HIG',
 'MCHP',
 'GPN',
 'MO',
 'SRE',
 'WELL',
 'TSCO',
 'NEM',
 'REG',
 'GM',
 'K',
 'VTRS',
 'BXP',
 'UHS',
 'JCI',
 'AKAM',
 'IEX',
 'ANSS',
 'V',
 'LUV']

# StandardScaler

In [7]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler

# google drive
from google.colab import drive
drive.mount('/content/drive') 

# ROOT
read = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/dataset/"

# save
save = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/dataset/Sampled_30_dataset/rescaled_data/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
sampled_list=['^GSPC']
for stock in sampled_list:
  print(stock)
  # read
  df = pd.read_csv(read+stock+'_week_features.csv')
  # drop inf and nan
  df = df.replace([np.inf,-np.inf],np.nan).dropna()

  cut = ['成交量均值','成交金额均值','上周涨跌幅','涨跌幅均值','股价波动程度','开收盘价方差','未来一周的收益率']

  # cols to rescale
  std_df = df[cut]

  # cut
  df = df[['name','stock','start_time','end_time','days']].reset_index()

  # rescaler
  scaler = StandardScaler()
  scaler.fit(std_df)
  result = pd.DataFrame(scaler.transform(std_df), columns=cut)

  # concat
  df = pd.concat([df[['name','stock','start_time','end_time','days']], result], axis=1)

  # save
  df.to_csv(save+stock+'_week_features_rescaled.csv')

^GSPC


In [ ]:
a = pd.DataFrame(scaler.transform(std_df), columns=['成交量均值','成交金额均值','上周涨跌幅','涨跌幅均值','股价波动程度','开收盘价方差','未来一周的收益率'])
a

,成交量均值,成交金额均值,上周涨跌幅,涨跌幅均值,股价波动程度,开收盘价方差,未来一周的收益率
0,-0.372663,-0.448707,-0.037860,0.365111,-0.557650,-0.381331,-0.018511
1,-0.399613,-0.508696,0.387030,0.460150,-0.635223,-0.503229,0.609192
2,0.282212,0.601274,0.387511,1.223114,1.523516,1.322448,0.069606
3,-0.271594,-0.245482,1.297821,0.213624,-0.887959,-0.531587,0.668596
4,-0.434147,-0.473104,0.226223,0.678275,-0.107521,-0.257258,0.920162
...,...,...,...,...,...,...,...
293,-0.434759,-0.870276,-1.068083,0.023026,-0.405714,-0.463917,-0.400224
294,-0.562784,-1.041082,0.023898,-0.425020,-0.586651,-0.479894,0.856012
295,-0.345209,-0.764908,-0.451713,1.227379,-0.005882,-0.410905,-1.817783
296,0.064190,-0.324167,1.039057,-2.343049,1.620589,0.238612,-1.477891


In [ ]:
df = df.replace([np.inf,-np.inf],np.nan).dropna()
df = df[['name','stock','start_time','end_time','days']].reset_index()

In [ ]:
pd.concat([df[['name','stock','start_time','end_time','days']], a], axis=1)

,name,stock,start_time,end_time,days,成交量均值,成交金额均值,上周涨跌幅,涨跌幅均值,股价波动程度,开收盘价方差,未来一周的收益率
0,Southwest Airlines Company,LUV,2017-01-09 00:00:00.000000000,2017-01-15 23:59:59.999999999,5,-0.372663,-0.448707,-0.037860,0.365111,-0.557650,-0.381331,-0.018511
1,Southwest Airlines Company,LUV,2017-01-16 00:00:00.000000000,2017-01-22 23:59:59.999999999,4,-0.399613,-0.508696,0.387030,0.460150,-0.635223,-0.503229,0.609192
2,Southwest Airlines Company,LUV,2017-01-23 00:00:00.000000000,2017-01-29 23:59:59.999999999,5,0.282212,0.601274,0.387511,1.223114,1.523516,1.322448,0.069606
3,Southwest Airlines Company,LUV,2017-01-30 00:00:00.000000000,2017-02-05 23:59:59.999999999,5,-0.271594,-0.245482,1.297821,0.213624,-0.887959,-0.531587,0.668596
4,Southwest Airlines Company,LUV,2017-02-06 00:00:00.000000000,2017-02-12 23:59:59.999999999,5,-0.434147,-0.473104,0.226223,0.678275,-0.107521,-0.257258,0.920162
...,...,...,...,...,...,...,...,...,...,...,...,...
293,Southwest Airlines Company,LUV,2022-08-22 00:00:00.000000000,2022-08-28 23:59:59.999999999,5,-0.434759,-0.870276,-1.068083,0.023026,-0.405714,-0.463917,-0.400224
294,Southwest Airlines Company,LUV,2022-08-29 00:00:00.000000000,2022-09-04 23:59:59.999999999,5,-0.562784,-1.041082,0.023898,-0.425020,-0.586651,-0.479894,0.856012
295,Southwest Airlines Company,LUV,2022-09-05 00:00:00.000000000,2022-09-11 23:59:59.999999999,4,-0.345209,-0.764908,-0.451713,1.227379,-0.005882,-0.410905,-1.817783
296,Southwest Airlines Company,LUV,2022-09-12 00:00:00.000000000,2022-09-18 23:59:59.999999999,5,0.064190,-0.324167,1.039057,-2.343049,1.620589,0.238612,-1.477891


# concat

In [ ]:
import pandas as pd
import numpy as np
import os

# google drive
from google.colab import drive
drive.mount('/content/drive') 

# ROOT
root = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/dataset/Sampled_30_dataset/rescaled_data"

# save
save = "/content/drive/MyDrive/BDT SEM 1/MSBD 5001/Data Preprocessing/dataset/Sampled_30_dataset"

os.chdir(root)
os.listdir(root)

dirs = os.listdir()
dirs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['AES_week_features_rescaled.csv',
 'DRI_week_features_rescaled.csv',
 'LVS_week_features_rescaled.csv',
 'ZION_week_features_rescaled.csv',
 'TJX_week_features_rescaled.csv',
 'KMX_week_features_rescaled.csv',
 'DVA_week_features_rescaled.csv',
 'PYPL_week_features_rescaled.csv',
 'INTU_week_features_rescaled.csv',
 'ADI_week_features_rescaled.csv',
 'HIG_week_features_rescaled.csv',
 'MCHP_week_features_rescaled.csv',
 'GPN_week_features_rescaled.csv',
 'MO_week_features_rescaled.csv',
 'SRE_week_features_rescaled.csv',
 'WELL_week_features_rescaled.csv',
 'TSCO_week_features_rescaled.csv',
 'NEM_week_features_rescaled.csv',
 'REG_week_features_rescaled.csv',
 'GM_week_features_rescaled.csv',
 'K_week_features_rescaled.csv',
 'VTRS_week_features_rescaled.csv',
 'BXP_week_features_rescaled.csv',
 'UHS_week_features_rescaled.csv',
 'JCI_week_features_rescaled.csv',
 'AKAM_week_features_rescaled.csv',
 'IEX_week_features_rescaled.csv',
 'ANSS_week_features_rescaled.csv',
 'V_week_featur

In [ ]:
dfs=[]
for file in dirs:
  df = pd.read_csv(root+'/'+file, index_col=0)
  dfs.append(df)

df_all = pd.concat(dfs, ignore_index=False)

In [ ]:
df_all.to_csv(save+'/'+'30_stocks_weekly_features.csv')

In [ ]:
df_all[['name','stock']].drop_duplicates(subset=None,keep='first',inplace=False).to_csv(save+'/'+'selected_30_stocks.csv')